# Example usage of CoReMultiHeadAttention
Note: this is a fairly stupid architecture which is only for illustration purpose.

In [1]:
import os
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from pytorch_lightning import Trainer, LightningModule
from modules import CoReMultiheadAttention

In [2]:
class CoMhaExample(LightningModule):
    def __init__(self):
        super().__init__()
        # mnist images are (1, 28, 28) (channels, height, width)
        self.co_mha = CoReMultiheadAttention(
            embed_dim=28,
            num_heads=2,
            dropout=0.0,
            bias=True,
            add_bias_kv=False,
            add_zero_attn=False,
            kdim=None,
            vdim=None,
            batch_first=True,
            device=None,
            dtype=None,
            sequence_len=28,
        )
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        
        # When using forward_step(s) to train, we need to clean_state between backward invocations.
        # Otherwise, we will try to backward through the graph a second time 
        self.co_mha.clean_state() 

        # (b, 1, 28, 28) -> (b, 28, 28)
        x = x.view(-1, 28, 28)
        x = self.co_mha.forward_steps(x, x, x) # or forward
        x = F.relu(x)
        x = x.reshape(-1, 28 * 28)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters())

In [3]:
# transforms
# prepare transforms standard to MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# data
mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
mnist_train = DataLoader(mnist_train, batch_size=64)

mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
mnist_test = DataLoader(mnist_test, batch_size=64)

/Users/au478108/opt/miniconda3/envs/continual-transformers/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# Fit the model
model = CoMhaExample()
trainer = Trainer(max_epochs=1)
trainer.fit(model, mnist_train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name   | Type                   | Params
--------------------------------------------------
0 | co_mha | CoReMultiheadAttention | 3.2 K 
1 | fc     | Linear                 | 7.9 K 
--------------------------------------------------
11.1 K    Trainable params
0         Non-trainable params
11.1 K    Total params
0.044     Total estimated model params size (MB)
/Users/au478108/opt/miniconda3/envs/continual-transformers/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]